In [18]:
path='/mnt/dev3-1/apps1/tk/codon_harmony/K0001_K0002'
!cd $path
!pwd

/mnt/dev3-1/apps2/tk/codon_harmony/K0001_K0002


In [19]:
input_file_name = 'input_K0001_K0002.fasta'
output_file_name = 'output_K0001_K0002.fasta'

In [20]:
codon_harmony_cmd = "codon_harmony --input " + input_file_name + " --output " \
+ output_file_name + " --host 9606 --verbose 1 --remove-splice-sites --no-remove-start-sites \
--cycles 10 --max-relax 0.2"
codon_harmony_cmd

'codon_harmony --input input_K0001_K0002.fasta --output output_K0001_K0002.fasta --host 9606 --verbose 1 --remove-splice-sites --no-remove-start-sites --cycles 10 --max-relax 0.2'

In [4]:
## Run codon_harmony
import subprocess
process = subprocess.Popen(codon_harmony_cmd, shell=True,
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
stdout, stderr
process.wait()

0

In [5]:
def check_integrity(file1,file2):
    fasta_obj1=Fasta(file1)
    fasta_obj2=Fasta(file2)
    
    TrueFalse = []
    for line in fasta_obj1.keys():
        integrity = fasta_obj1[line][0:].seq == Seq(fasta_obj2[line][0:].seq).translate()
        print(fasta_obj1[line].name)
        print("protein_seq == opt_dna_seq : ",integrity)
        TrueFalse.append(integrity)
    print('Integrity: ', TrueFalse)
    return [i for i,x in enumerate(TrueFalse) if TrueFalse[i]==False]

In [6]:
def check_alignment(fasta_obj1, fasta_obj2, failed_pairs):
    
    for failed_pair in failed_pairs:

        failed_protein=fasta_obj1[failed_pair][0:].seq
        failed_opt_dna=Seq(fasta_obj2[failed_pair][0:].seq).translate()
        alignments = pairwise2.align.globalxx(failed_protein, failed_opt_dna)
        print('\n >>Failed pair: ',fasta_obj1[failed_pair].name)
        for alignment in alignments: 
            print(format_alignment(*alignment)) 

In [7]:
from Bio.Seq import Seq
from Bio import pairwise2
from Bio.pairwise2 import format_alignment 
from Bio.Alphabet import generic_dna
from pyfaidx import Fasta

protein = Fasta(input_file_name)
opt_dna = Fasta(output_file_name)

failed_pairs=check_integrity(input_file_name, output_file_name)

check_alignment(protein, opt_dna, failed_pairs)

K0001_TMG1
protein_seq == opt_dna_seq :  True
K0001_TMG2
protein_seq == opt_dna_seq :  True
K0001_TMG3
protein_seq == opt_dna_seq :  True
K0001_TMG4
protein_seq == opt_dna_seq :  True
K0001_TMG5
protein_seq == opt_dna_seq :  True
K0002_TMG1
protein_seq == opt_dna_seq :  True
K0002_TMG2
protein_seq == opt_dna_seq :  True
K0002_TMG3
protein_seq == opt_dna_seq :  True
K0002_TMG4
protein_seq == opt_dna_seq :  True
K0002_TMG5
protein_seq == opt_dna_seq :  True
K0002_TMG6
protein_seq == opt_dna_seq :  True
K0002_TMG7
protein_seq == opt_dna_seq :  True
Integrity:  [True, True, True, True, True, True, True, True, True, True, True, True]


In [8]:
## if there is wrong, rerun codon_harmony for the wrong pair and merge it with previous result that was correct.
if failed_pairs:
    ## collect correct output
    f = open("first_round_correct_output.fasta", "w")
    for i in range(len(protein.keys())):

        if i in failed_pairs:
            pass
        else:
            f.write(">"+opt_dna[i].name+"\n")
            f.write(opt_dna[i][0:].seq+"\n")
    f.close()
    
    ## re-Run codon_harmony
    failed_pair_name=protein[failed_pair].name
    input_rerun_file_name = 'input_rerun_K0001_K0002.fasta'
    output_rerun_file_name = 'output_rerun_K0001_K0002.fasta'
    reform_output_rerun_file_name = 'reform_output_rerun_K0001_K0002.fasta'

    codon_harmony_cmd = "codon_harmony --input " + input_rerun_file_name + " --output " \
    + output_rerun_file_name + " --host 9606 --verbose 1 --remove-splice-sites --no-remove-start-sites --cycles 10 --max-relax 0.2"

    rerun_cmd = "cat input_K0001_K0002.fasta | grep -A1 " + failed_pair_name + " > " + input_rerun_file_name + "\n"
    rerun_cmd = rerun_cmd + codon_harmony_cmd

    process = subprocess.Popen(rerun_cmd, shell=True,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    stdout, stderr
    process.wait()

    fasta_obj1 = Fasta(input_rerun_file_name)
    fasta_obj2 = Fasta(output_rerun_file_name)

    rerun_failed_pairs=check_integrity(input_rerun_file_name, output_rerun_file_name)
    check_alignment(fasta_obj1, fasta_obj2, rerun_failed_pairs)

    ## merge outputs from first round and second round
    reform_f=open(reform_output_rerun_file_name, "w")
    for i in range(len(fasta_obj2.keys())):    
        reform_f.write("\n>"+fasta_obj2[i].name+"\n")
        reform_f.write(fasta_obj2[i][0:].seq+"\n")
    reform_f.close()

    if not rerun_failed_pairs:

        final_output_cmd = "cat " + "first_round_correct_output.fasta" + " " + reform_output_rerun_file_name + " > final_output.fasta"
    #     print(final_output_cmd)
        process = subprocess.Popen(final_output_cmd, shell=True)
        process.wait()
